In [1]:
%pylab

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
#Added all Libraries
from spectral import *
import numpy as np
import PIL
import matplotlib
import matplotlib.pyplot as plt
import mpmath as mp

In [3]:
#image stored in arr
img = aviris.open('hawaii_sc01.raw')
print img
arr = img.load()
block = []
for i in range(224):
    data = arr[:10,:10,i].astype(np.float32)*1000000
    block.append(data)
print "process complete"
varblock = []
for hg in range(224):
    yog = np.var(block[hg])
    varblock.append(yog)
variance = np.average(varblock)
##### considering 40x40 matrix
block = []
for i in range(224):
    data = arr[:40,0:40,i].astype(np.float32)*1000000
    block.append(data)
print "process complete"

	Data Source:   './hawaii_sc01.raw'
	# Rows:            512
	# Samples:         614
	# Bands:           224
	Interleave:        BIP
	Quantization:  16 bits
	Data format:     int16
process complete
process complete


In [4]:
#correlation
corr = []
for i in range(223):
    cor = np.corrcoef(block[i].reshape(-1), block[i+1].reshape(-1))[0][1]
    corr.append(cor)
print "process complete"

process complete


In [5]:
#Pre-Processing of the hyperspectral image, Hawaii. Using GOA technique
haha = corr.index(min(corr))
print haha

159


In [6]:
#greedy ordering algorithm
rblock = [] #reference block
block1 = [] #all blocks
block1 = block
rindex = [] #contains all index
ifin = []
ifin1 = []


rindex.append(haha)
rblock.append(block1[haha])

for gh in range(len(block)):
        ifin.append(gh) #ifin will conatin the number od indices
        
for jk in range(224): 
    corgoa = [] #conatins correlation of greedy ordering algorithm
    Ref = rindex[jk] #Ref will contain the last rindex    
    block1.pop(Ref) #Removes that index from the block
    yoyo = ifin.pop(Ref) # saves it to a temp variable yoyo
    ifin1.append(yoyo) #ifin1 will conatin the final rearranged indexes
    for ijk in range(len(block1)):
        corr2 = np.corrcoef(rblock[jk].reshape(-1), block1[ijk].reshape(-1))[0][1] #calculate correlation coefficient with respect to the reference block
        corgoa.append(corr2)
    if len(corgoa) !=0:
        indexi = corgoa.index(max(corgoa)) #indexi will contain the index of maximum correlation w.r.t to the rblock
        rblock.append(block1[indexi]) #add the corresponding block to the rblock
        rindex.append(indexi) 
    del(corgoa)
    del(Ref)
print "Process Complete"

Process Complete


In [7]:
corrfin = []
for i in range(223):
    corfin = np.corrcoef(rblock[i].reshape(-1), rblock[i+1].reshape(-1))[0][1]
    corrfin.append(corfin)
print "process complete"

process complete


In [8]:
plt.figure(12)
plt.plot(range(0,223),corrfin, 'k')
plt.xlabel('Band Number')
plt.ylabel('Correlation Coefficient')
plt.title('Correlation Coefficient vs Band Number after GOA')
plt.show()

In [9]:
len(rblock[0])

40

In [10]:
#normalizing each block
rblockn = []
maxblock = []
minblock = []
for q in range(len(rblock)):
    xc = rblock[q]
    maxxc = np.max(xc)
    maxblock.append(maxxc)
    minxc = np.min(xc)
    minblock.append(minxc)
    xcfin = []
    for xci in range(len(xc)):
        for xcj in range(len(xc)):
            xcres = (xc[xci][xcj] - minxc)/(maxxc - minxc) 
            xcfin.append(xcres)
    xcfin1 = np.reshape(xcfin,(40,-1))
    rblockn.append(xcfin1)
print 'Process Complete'

Process Complete


In [11]:
def raster(arr):
    a = arr
    xx = []
    for i in range(10):
        zz = []
        for j in range(10):
            x = float(a[i][j])
            zz.append(x)
        xx.extend(zz)
        del zz
    return xx

In [12]:
#imagescanning
finlist = []
for i in range(224):
    rblockyo = rblockn[i]
    xx = rblockyo[:10,:10]
    yy = raster(xx)
    finlist = finlist + yy

In [13]:
len(finlist)

22400

In [14]:
finlist = list(finlist)

In [15]:
plt.figure(29)
plt.plot(range(500,1000),finlist[500:1000], 'k')
plt.xlabel('Band')
plt.ylabel('Correlation')
plt.title('Correlation vs band')
plt.show()

In [16]:
from audiolazy import lpc

In [17]:
y = 40 #frame size

In [18]:
finli = map (lambda x: finlist[y*x:(x+1)*y], range (len(finlist)/y)) 

In [19]:
reslpcfin = []
errorframe = []
coefficient = []
lpcfunction = []
lpcerror = []
errorfull = []
for ijk in range(len(finlist)/y):
    lpcframe = lpc(finli[ijk],order=3)
    lpcfunction.append(lpcframe)
    lpcerror.append(lpcframe.error)
    reslpc = list(lpcframe(finli[ijk]))
    reslpcfin.append(reslpc)
    coefficient.append(lpcframe.numerator)
    errorfr = []
    for ijkl in range(len(reslpc)):
        err = finli[ijk][ijkl] - reslpc[ijkl]
        errorfr.append(err)
        errorfull.append(err)
    errorframe.append(errorfr)
    del(errorfr)
    del(reslpc)
    del(lpcframe)
print "process complete"

process complete


In [20]:
#lpcjai1 = lpc(finlist, order=3).plot().show()

In [21]:
from sklearn import cluster

In [22]:
abcd = np.array(errorframe)

In [23]:
K_means = cluster.KMeans(n_clusters=128,max_iter=1,n_init =1, init='k-means++',tol = '1e-5')

In [24]:
fincodebook1 = K_means.fit(abcd)

In [25]:
codebook = K_means.cluster_centers_.squeeze()
huffcodebook = list(K_means.labels_)

In [26]:
dist = K_means.inertia_
print dist

28.0937930457


In [27]:
#print codebook
#print huffcodebook

In [28]:
from heapq import heappush, heappop, heapify
from collections import defaultdict
 
def encode(symb2freq):
    """Huffman encode the given dict mapping symbols to weights"""
    heap = [[wt, [sym, ""]] for sym, wt in symb2freq.items()]
    heapify(heap)
    while len(heap) > 1:
        lo = heappop(heap)
        hi = heappop(heap)
        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]
        heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
    return sorted(heappop(heap)[1:], key=lambda p: (len(p[-1]), p))
 
txt = "this is an example for huffman encoding"
symb2freq = defaultdict(int)
for ch in huffcodebook:
    symb2freq[ch] += 1
# in Python 3.1+:
# symb2freq = collections.Counter(txt)
huff = encode(symb2freq)
print "Symbol\tWeight\tHuffman Code"
huffsymbol = []
hufffreq = []
for p in huff:
    huffsymbol.append(p[1])
    hufffreq.append(symb2freq[p[0]])
    #print "%s\t%s\t%s" % (p[0], symb2freq[p[0]], p[1])

Symbol	Weight	Huffman Code


In [29]:
lenhuff = []
for i in range(len(huffsymbol)):
    yo = huffsymbol[i]
    x = str(yo)
    ya = x.lstrip('0')
    lenhuff.append(len(ya)*hufffreq[i])
print sum(lenhuff)

3070


In [30]:
len(lenhuff)

128

In [31]:
cr = float(len(finlist)*16)/sum(lenhuff)
print cr

116.74267101


In [32]:
def huffall(y):
    finli = map (lambda x: finlist[y*x:(x+1)*y], range (len(finlist)/y)) 
    print len(finli)
    global huffcode, lenhuff
    huffcode = []
    lenhuff = []
    for sym in finli:
        frequency = defaultdict(int)
        for symbol in sym:
            frequency[symbol] += 1

        huff = encode(frequency)
        #print "Symbol".ljust(10) + "Weight".ljust(10) + "Huffman Code"
        repeat = frequency.values()

        for i in range(len(huff)):
            p = huff[i]
            j = repeat[i]
            for k in range(j):
                x = str(p[1])
                ya = x.lstrip('0')
                lenyz = len(ya)
                #print  p[0] , str(frequency[p[0]]).ljust(10) , ya
                lenhuff.append(lenyz)
                huffcode.append(p[1])
        del frequency
    print 'process complete'
    print "The length of Huffcode", len(huffcode)
    cr = float(len(finlist)*16)/sum(lenhuff)
    print "The compression ratio for frame size", y, "is", cr

In [33]:
#BPP
bpp = sum(lenhuff)/22400.00
print bpp

0.137053571429


In [34]:
#huffcodebook - vector de-quantization
errordecode = []
for ik in huffcodebook:
    abcde = list(codebook[ik])
    errordecode.append(abcde)

In [35]:
len(errordecode)

560

In [36]:
print coefficient[0]
print "---"
print lpcerror[0]
print "---"
print lpcfunction[0] #h
print "---"
print finli[0] #x
print "---"
print reslpcfin[0] #y
print "---"
print errordecode[0] #e


[1, -0.5004631234896397, -0.22660480902576108, -0.1981380961527308]
---
3.45748995332
---
1 - 0.500463 * z^-1 - 0.226605 * z^-2 - 0.198138 * z^-3
---
[0.7500005820762427, 0.24999941792375732, 0.5000011641524853, 0.7500005820762427, 0.7500005820762427, 1.0, 0.24999941792375732, 0.5000011641524853, 0.5000011641524853, 0.5000011641524853, 0.5000011641524853, 0.5000011641524853, 0.5000011641524853, 0.7500005820762427, 0.5000011641524853, 0.24999941792375732, 0.5000011641524853, 0.5000011641524853, 0.5000011641524853, 0.5000011641524853, 0.24999941792375732, 1.0, 0.7500005820762427, 0.5000011641524853, 0.5000011641524853, 0.7500005820762427, 0.7500005820762427, 0.24999941792375732, 0.5000011641524853, 0.24999941792375732, 0.5000011641524853, 0.7500005820762427, 0.7500005820762427, 0.5000011641524853, 0.7500005820762427, 0.7500005820762427, 0.7500005820762427, 0.7500005820762427, 0.7500005820762427, 1.0]
---
[0.7500005820762427, -0.12534821600143717, 0.20493193591690406, 0.2945136799144009, 

In [37]:
from scipy.signal import lfilter

In [38]:
estx = []

In [39]:
estx.append(reslpcfin[0][0]+errordecode[0][0])

In [40]:
estx

[0.75000058207624265]

In [41]:
esty = list(lpcfunction[0](estx))

In [42]:
esty

[0.75000058207624265]

In [43]:
len(lpcfunction)

560

In [44]:
estx = []
esty = []
for iq in range(len(errordecode)):
    estx1 = []
    esty1 = []
    for iw in range(len(errordecode[iq])):
        estx1.append(reslpcfin[iq][iw]+errordecode[iq][iw])
        esty1 = list(lpcfunction[iq](estx1))
    esty.append(esty1)
    estx.append(estx1)

In [45]:
print estx[0]
print finli[0]

[0.75000058207624265, 0.24999941792375732, 0.5000011641524853, 0.75000058207624265, 0.75000058207624254, 1.0, 0.24999941792375735, 0.5000011641524853, 0.5000011641524853, 0.5000011641524853, 0.5000011641524853, 0.5000011641524853, 0.5000011641524853, 0.75000058207624265, 0.5000011641524853, 0.24999941792375729, 0.5000011641524853, 0.5000011641524853, 0.5000011641524853, 0.5000011641524853, 0.24999941792375732, 1.0, 0.75000058207624265, 0.5000011641524853, 0.5000011641524853, 0.75000058207624265, 0.75000058207624265, 0.24999941792375735, 0.5000011641524853, 0.24999941792375735, 0.5000011641524853, 0.75000058207624265, 0.75000058207624254, 0.5000011641524853, 0.75000058207624265, 0.75000058207624265, 0.75000058207624265, 0.75000058207624254, 0.75000058207624254, 1.0]
[0.7500005820762427, 0.24999941792375732, 0.5000011641524853, 0.7500005820762427, 0.7500005820762427, 1.0, 0.24999941792375732, 0.5000011641524853, 0.5000011641524853, 0.5000011641524853, 0.5000011641524853, 0.50000116415248

In [46]:
print esty[0]
print reslpcfin[0]

[0.75000058207624265, -0.12534821600143717, 0.20493193591690406, 0.29451367991440092, 0.21181587112872496, 0.35562934866457274, -0.56902113168324697, -0.00032282188454871297, -0.0050201467161278823, 0.086931942769773243, 0.037397072737087644, 0.037397072737087644, 0.037397072737087644, 0.28739649066084505, -0.08771841682771786, -0.26925574384686457, 0.11297931882589973, 0.094048670697415071, 0.086931942769773257, 0.037397072737087644, -0.21260467349164033, 0.66251256338018594, 0.093817109491561412, -0.15148568750548064, -0.11832281503167946, 0.23786208195407355, 0.16228100109603946, -0.39437123341167002, 0.056328248470675468, -0.20548748423808438, 0.21204859756535671, 0.34404808862117242, 0.21181587112872496, -0.14436948718294199, 0.18121101159884934, 0.11274659238926801, 0.10562993074081531, 0.056095522034043721, 0.056095522034043777, 0.30609493995780124]
[0.7500005820762427, -0.12534821600143717, 0.20493193591690406, 0.2945136799144009, 0.21181587112872508, 0.35562934866457274, -0.56

In [47]:
estxfin = []
for yi in estx:
    for yu in yi:
        estxfin.append(yu)

In [48]:
y = 100
estxfin1 = map (lambda x: estxfin[y*x:(x+1)*y], range (len(finlist)/y)) 

In [49]:
blockfin = []
for iji in estxfin1:
    bloc = np.reshape(iji,[10, 10])
    blockfin.append(bloc)

In [50]:
len(maxblock)

224

In [51]:
finblockfin = []
for i in range(len(blockfin)):
    eega = blockfin[i]
    eegamax = np.max(eega)
    eegamin = np.min(eega)
    oldmax = maxblock[i]
    oldmin = minblock[i]
    eegvalue = []
    for j in range(len(eega)):
        for k in range(len(eega)):
            ee = ((eega[j][k] - eegamin)*(oldmax - oldmin))/(eegamax - eegamin)
            eee = ee  + oldmin
            eegvalue.append(eee)
    fblock = np.reshape(eegvalue, [10,10])
    finblockfin.append(fblock)
print "Process Complete"

Process Complete


In [52]:
finblockfin[0]

array([[ 8606.63791807,  8419.91264032,  8513.27571394,  8606.63791807,
         8606.63791807,  8700.00012219,  8419.91264032,  8513.27571394,
         8513.27571394,  8513.27571394],
       [ 8513.27571394,  8513.27571394,  8513.27571394,  8606.63791807,
         8513.27571394,  8419.91264032,  8513.27571394,  8513.27571394,
         8513.27571394,  8513.27571394],
       [ 8419.91264032,  8700.00012219,  8606.63791807,  8513.27571394,
         8513.27571394,  8606.63791807,  8606.63791807,  8419.91264032,
         8513.27571394,  8419.91264032],
       [ 8513.27571394,  8606.63791807,  8606.63791807,  8513.27571394,
         8606.63791807,  8606.63791807,  8606.63791807,  8606.63791807,
         8606.63791807,  8700.00012219],
       [ 8513.27571394,  8371.28428086,  8502.83041578,  8508.04415532,
         8530.33689224,  8527.52625049,  8505.04373516,  8401.16324889,
         8531.65679934,  8525.7720572 ],
       [ 8514.34102113,  8510.17523889,  8496.95226277,  8529.47803341,
   

In [53]:
rblock5 = []
for i in range(len(rblock)):
    a = rblock[i]
    ab = a[:10,:10]
    rblock5.append(ab)

In [54]:
psnr = []
mse = []
for i in range(224):
    aq = rblock5[i]
    aw = finblockfin[i]
    subtr = []
    for j in range(10):
        for k in range(10):
            sub = (aq[j][k] - aw[j][k])**2
            subtr.append(sub)
    mse1 = np.sum(subtr)/100.0
    mse.append(mse1)
    psnr1 = 10*np.log10(65536**2/mse1)
    psnr.append(psnr1)
print "process complete"

process complete


/Library/Python/2.7/site-packages/ipykernel/__main__.py:13: RuntimeWarning: divide by zero encountered in double_scalars


In [55]:
for i in range(20):
    a = psnr.index(inf)
    psnr[a] = 104

ValueError: inf is not in list

In [56]:
psnr1 = np.sum(psnr)/224.0

In [57]:
sdr = 10*np.log10(variance**2/dist)

In [58]:
plt.figure(80)
plt.plot(range(224),psnr, 'k')
plt.xlabel('Band Number')
plt.ylabel('PSNR (dB)')
plt.title('PSNR vs Band Number')
plt.show()

In [59]:
print 'cr = ', cr
print 'bpp= ', bpp
print 'sdr= ', sdr
print 'psnr= ', psnr1
print 'dist= ',dist

cr =  116.74267101
bpp=  0.137053571429
sdr=  120.34903298
psnr=  42.8363302544
dist=  28.0937930457


In [ ]:
variance